# Tightbinding simulation of a polaritonic topological insulator

In this notebook, we propose to recreate topological insulator proposed in this [article](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.114.116401).

The lattice described is a polariton honeycomb lattice with non-zero Zeeman splitting and Spin-orbit coupling. This 4-band model hosts a topologically non-trivial phase when the Zeeman splitting is stronger than the sublattice energy difference, which we propose to demonstrate here.

## Building the lattice

In [1]:
from tightbinding.geometry import Orbital, Site, Unitcell, Lattice

a = 2.4

plus_orbital = Orbital('+')
minus_orbital = Orbital('-')

a = 3.36 # Lattice constant used in the main article describing this work
a_site = Site('A', [0, a/2], [minus_orbital, plus_orbital])
b_site = Site('B', [0, -a/2], [minus_orbital, plus_orbital])

a1 = [-3**0.5/2 * a, 3/2 * a] # 1st lattice vector
a2 = [3**0.5/2 * a, 3/2 * a] # 2nd lattice vector
uc = Unitcell("honeycomb", [a1, a2])
uc.add_site([a_site, b_site])

# Creating a periodic lattice made of a single unit cell
honeycomb = Lattice('honeycomb', uc, (True, True))
honeycomb.add_unitcell((0, 0), update=True)
honeycomb.plot()

# Constructing the Hamiltonian

We are going to build the Bloch Hamiltonian

$$H(\bm{k}) = -\begin{bmatrix}
    -\epsilon_0/2-\epsilon_Z/2 & 0 & t \gamma(\bm{k}) & \delta t \gamma^+(\bm{k}) \\
    0 & -\epsilon_0/2+\epsilon_Z/2 & \delta t \gamma^-(\bm{k}) & t\gamma(\bm{k}) \\
    t\gamma^*(\bm{k}) & \delta t (\gamma^-(\bm{k}))^* & \epsilon_0/2-\epsilon_Z/2 & 0 \\
    \delta t (\gamma^+(\bm{k}))^* & t\gamma^*(\bm{k}) & 0 & \epsilon_0/2+\epsilon_Z/2
    \end{bmatrix}
$$

with $\epsilon_0/2$ the on-site energy difference, $\epsilon_Z$ the Zeeman splitting, $t$ the same-polarization hopping strength and $\delta t$ the opposite-polarization hopping strength and

$$\gamma(\bm{k}) = e^{-i\bm{k} \cdot \bm{d}_{1}}+e^{-i\bm{k} \cdot \bm{d}_{2}}+e^{-i\bm{k} \cdot \bm{d}_{3}} \\
    \gamma^{\pm}(\bm{k}) = e^{-i(\bm{k} \cdot \bm{d}_{1} \mp 2 \phi_1) }+e^{-i(\bm{k} \cdot \bm{d}_{2} \mp 2 \phi_2) }+e^{-i(\bm{k} \cdot \bm{d}_{3} \mp 2 \phi_3) }
$$
where $\phi_i$ is the angle between the link $i$ and the x-axis.


In [2]:
## Constructing the Hamiltonian

from tightbinding.hamiltonian import Energy, Hopping, Hamiltonianbuilder
import numpy as np

kl = 1.2
n_k = 151

parameters = {
    "kx": np.linspace(-kl, kl, n_k),
    "ky": np.linspace(-kl, kl, n_k),
    "t": -1, # hopping strength
    "zeeman": np.linspace(-1,1, 5), # zeeman splitting
    "delta_t": np.linspace(0,0.5, 5), # spin-orbit induced hopping
    "eps0": np.linspace(-1,1, 5), # on-site energy difference
}

# Specifying the on-site energies
onsite = Energy(honeycomb)
onsite.set_energy("-eps0/2", nsite='A')
onsite.set_energy("eps0/2", nsite='B')
onsite.add_energy("-zeeman/2", norbital='-')
onsite.add_energy("zeeman/2", norbital='+')

# Specifying the hoppings
links = [(0,0), (1,0), (0,1)]
links_angle = [-np.pi/2 + 2*np.pi/3 * k for k in range(3)]

# same-polarization hoppings
hoppings = [Hopping(honeycomb, 't', 'A_+', 'B_+', link) for link in links]
hoppings += [Hopping(honeycomb, 't', 'A_-', 'B_-', link) for link in links]
# cross-polarization hoppings
hoppings += [Hopping(honeycomb, f'delta_t * np.exp(1j * 2 * {theta})', 'A_-', 'B_+', link) for link, theta in zip(links, links_angle)]
hoppings += [Hopping(honeycomb, f'delta_t * np.exp(-1j * 2 * {theta})', 'A_+', 'B_-', link) for link, theta in zip(links, links_angle)]

Hamiltonian = Hamiltonianbuilder(honeycomb, parameters, ['kx', 'ky'])
Hamiltonian.set_on_site_energies(onsite)
Hamiltonian.add_couplings(hoppings)
Hamiltonian.build()

Now that the Hamiltonian is build, let's look at its band structure.

# Band structure

In [3]:
eigva, eigve = Hamiltonian.eigh()

In [4]:
from tightbinding.plotting import plot_bands, plot_bands_3D

plot_bands(eigva, 'kx')

BokehModel(combine_events=True, render_bundle={'docs_json': {'67e25bbf-3edc-431f-b516-d3be315c7f07': {'version…

We can observe multiple features in this band structure. First, when $|\epsilon_0|$ and $|\epsilon_Z|$ are diffrent, the band structure is gapped. Second, when these two terms are zero, a trigonal warping of the bands near the K and K' is visible when $\delta t \neq 0$. Let's look at this feature more closely in 3D.  

In [5]:
plot_bands_3D(eigva, escale = 5)

    'data': [{'colorscale': [[0.0, '#636EFA'], [1.0, '#636EFA']],
              …

Artifacts in the band structure appear in the features of the trigonal warping. This is due to the low resolution of these sharp features. We will zoom on them latter. Now, let's look at the sublattice localization and polarization of the eigenvectors.

In [6]:
%reload_ext autoreload
%autoreload 2

# first we determine the index of each orbital
i_ap = uc.orbitals_index["A_+"]
i_am = uc.orbitals_index["A_-"]
i_bp = uc.orbitals_index["B_+"]
i_bm = uc.orbitals_index["B_-"]

sublattice_localization = (abs(eigve.sel(component = i_ap))**2 + abs(eigve.sel(component = i_am))**2)*2 - 1 # equal to 1 if the fully localized on the A sublattice
polarization_degree = (abs(eigve.sel(component = i_ap))**2 + abs(eigve.sel(component = i_bp))**2)*2 - 1 # equal to 1 if the fully + polarized

energy_contour = eigva.sel(band = 2) - eigva.sel(band = 1)

from tightbinding.plotting import plot_eigenvectors
# Let's define our own plotting templates
template_sublattice = {
    "colorbar":{
        "tickvals": [-1,1],
        "ticktext": ["B","A"],
        },
    "cmap": "PiYG",
    "zmin":-1,
    "zmax":1,   
}

template_polarization = {
    "colorbar":{
        "tickvals": [-1,1],
        "ticktext": ["-","+"],
        },
    "cmap": "RdBu_r",
    "zmin":-1,
    "zmax":1,   
}

plot_eigenvectors(
    plots = [[sublattice_localization, polarization_degree]],
    eigvas = [[energy_contour, energy_contour]],
    templates=[[template_sublattice, template_polarization]],
    titles=[['Sublattice localization', 'Degree of polarization']]
)


    'data': [{'colorbar': {'len': 0.7,
                           'ticktext': [B…

Looking closely at the results for the bands 1 and 2 (inner bands), we can see that when $|\epsilon_0| > |\epsilon_Z|$, the sublattice localization has the same sign for both valleys, and the polarization degree is opposite. When $|\epsilon_0| < |\epsilon_Z|$ however, the opposite is true. This is typical of a band inversion, suggesting a topological transition. Let's compute the Berry curvature to be sure of it.

## Berry curvature

In [7]:
from tightbinding.topology import berry_curvature

berry = berry_curvature(eigve)

plot_eigenvectors(
    plots = [[berry.sel(band = 1), berry.sel(band = 2)]],
    eigvas = [[energy_contour, energy_contour]],
    templates=[['symmetric', 'symmetric']],
    titles=[['band 1', 'band 2']]
)


    'data': [{'colorbar': {'len': 0.7, 'tickformat': '.0e', 'x': np.float64(0.42…

This computation confirms a topological transition, as the berry poles change signs only in one of the valleys at the transition. To be sure of that, let's finally compute the Chern numbers for each band. To do so, we are going to redefine the parameters of our HamiltonianBuilder object, to only sample a single unit cell, then we are going to perform the integration. Here is ho to do it.

In [8]:
## Constructing the Hamiltonian

n_k = 51 # coarse array to increase the number of parameter points

parameters = {
    "ka1s": np.linspace(0, 1, n_k), # Here, we sample points along the lattice vectors, and not kx or ky
    "ka2s": np.linspace(0, 1, n_k),
    "t": -1, # hopping strength
    "zeeman": np.linspace(-1,1, 11), # zeeman splitting
    "delta_t": np.linspace(0,0.5, 11), # spin-orbit induced hopping
    "eps0": np.linspace(-1,1, 11), # on-site energy difference
}

# Then we create a new HamiltonianBuilder object
Hamiltonian_uc = Hamiltonianbuilder(honeycomb, parameters, ['kx', 'ky']) # The reciprocal coordinates are not in parameters, we need to build them ourselves.

a1s = np.array([3**0.5,-1])*2*np.pi/3/a
a2s = np.array([3**0.5, 1])*2*np.pi/3/a
# This can be done easily using the broadcasting rules of xarrays
kx = Hamiltonian_uc.Hamiltonian.ka1s * a1s[0] + Hamiltonian_uc.Hamiltonian.ka2s * a2s[0]
ky = Hamiltonian_uc.Hamiltonian.ka1s * a1s[1] + Hamiltonian_uc.Hamiltonian.ka2s * a2s[1]

# The arrays kx and ky are two-dimensional arrays containing the values of kx and ky for each point of the grid.

We have created multi-dimensional coordinates, which we can pass to the Hamiltonian builder, using the dedicated function.

In [9]:
Hamiltonian_uc.add_multidimensional_coord('kx', kx.data, ('ka1s','ka2s'))
Hamiltonian_uc.add_multidimensional_coord('ky', ky.data, ('ka1s','ka2s'))

Hamiltonian_uc.set_on_site_energies(onsite)
Hamiltonian_uc.add_couplings(hoppings)
Hamiltonian_uc.build()
eigva_uc, eigve_uc = Hamiltonian_uc.eigh()

In [10]:
%reload_ext autoreload
%autoreload 2

berry_uc = berry_curvature(eigve_uc, dims=['ka1s', 'ka2s'])
energy_contour_uc = eigva_uc.sel(band = 2) - eigva_uc.sel(band = 1)

# To plot the arrays correctly, we have to attach KX and KY
berry_uc = berry_uc.assign_coords({"kx":kx, "ky":ky})
energy_contour_uc = energy_contour_uc.assign_coords({"kx":kx, "ky":ky})

# plot_eigenvectors does not support multi-dimensionnal variables for plotting yet, so we plot along the ka1s and ka2s dimensions
plot_eigenvectors(
    plots = [[berry_uc.sel(band = 1), berry_uc.sel(band = 2)]],
    eigvas = [[energy_contour_uc, energy_contour_uc]],
    templates=[['symmetric', 'symmetric']],
    titles=[['band 1', 'band 2']],
    dims=['ka1s', 'ka2s']
)


    'data': [{'colorbar': {'len': 0.7, 'tickformat': '.0e', 'x': np.float64(0.42…

Now that we have the berry curvature, we can compute the Chern number for the gap.

In [11]:
chern = (berry_uc.sel(band=[0,1]).sum(["ka1s", "ka2s", 'band']) / 2 / np.pi).astype(int)

print(chern.sel(delta_t = 0.25, zeeman = 0.5, eps0 = 0, method = 'nearest'))
print(chern.sel(delta_t = 0.25, zeeman = 0.5, eps0 = 1, method = 'nearest'))

<xarray.DataArray 'value' ()> Size: 8B
array(2)
Coordinates:
    zeeman   float64 8B 0.4
    delta_t  float64 8B 0.25
    eps0     float64 8B 0.0
<xarray.DataArray 'value' ()> Size: 8B
array(0)
Coordinates:
    zeeman   float64 8B 0.4
    delta_t  float64 8B 0.25
    eps0     float64 8B 1.0


## Trigonal warping

Now, let's go back to the trigonal warping. Using the 'plot_eigenvector' function, it is really easy to explore the impact of the different parameters on the eigenvector structure around the K and K' points.

In [12]:
## Constructing the Hamiltonian

n_k = 101 
K = np.array([4*np.pi/3**1.5/a, 0])
dk = 0.4

parameters_K = {
    "kx": np.linspace(K[0]-dk, K[0]+dk, n_k), # Here, we sample points along the lattice vectors, and not kx or ky
    "ky": np.linspace(-dk, dk, n_k),
    "t": -1, # hopping strength
    "zeeman": np.linspace(-1,1, 5), # zeeman splitting
    "delta_t": np.linspace(0,0.5, 11), # spin-orbit induced hopping
    "eps0": np.linspace(-1,1, 5), # on-site energy difference
}

parameters_Kp = {
    "kx": np.linspace(-K[0]-dk, -K[0]+dk, n_k), # Here, we sample points along the lattice vectors, and not kx or ky
    "ky": np.linspace(-dk, dk, n_k),
    "t": -1, # hopping strength
    "zeeman": np.linspace(-1,1, 5), # zeeman splitting
    "delta_t": np.linspace(0,0.5, 11), # spin-orbit induced hopping
    "eps0": np.linspace(-1,1, 5), # on-site energy difference
}


# We create two HamiltonianBuilders
Hamiltonian_K = Hamiltonianbuilder(honeycomb, parameters_K, ['kx', 'ky'])
Hamiltonian_K.set_on_site_energies(onsite)
Hamiltonian_K.add_couplings(hoppings)
Hamiltonian_K.build()
eigva_K, eigve_K = Hamiltonian_K.eigh_parallel()
berry_K = berry_curvature(eigve_K)

Hamiltonian_Kp = Hamiltonianbuilder(honeycomb, parameters_Kp, ['kx', 'ky'])
Hamiltonian_Kp.set_on_site_energies(onsite)
Hamiltonian_Kp.add_couplings(hoppings)
Hamiltonian_Kp.build()
eigva_Kp, eigve_Kp = Hamiltonian_Kp.eigh_parallel()
berry_Kp = berry_curvature(eigve_Kp)


c:\Users\marti\miniconda3\envs\testenv\Lib\site-packages\dask\array\utils.py:105: ComplexWarning:

Casting complex values to real discards the imaginary part

c:\Users\marti\miniconda3\envs\testenv\Lib\site-packages\dask\array\utils.py:105: ComplexWarning:

Casting complex values to real discards the imaginary part



In [13]:
# Let's define a few quantities

from tightbinding.utils import angle

phases_K = [angle(eigve_K.sel(component = u)) for u in range(1,4)]
phases_Kp = [angle(eigve_Kp.sel(component = u)) for u in range(1,4)]

sublattice_localization_K = (abs(eigve_K.sel(component = i_ap))**2 + abs(eigve_K.sel(component = i_am))**2)*2 - 1 # equal to 1 if the fully localized on the A sublattice
polarization_degree_K = (abs(eigve_K.sel(component = i_ap))**2 + abs(eigve_K.sel(component = i_bp))**2)*2 - 1 # equal to 1 if the fully + polarized

sublattice_localization_Kp = (abs(eigve_Kp.sel(component = i_ap))**2 + abs(eigve_Kp.sel(component = i_am))**2)*2 - 1 # equal to 1 if the fully localized on the A sublattice
polarization_degree_Kp = (abs(eigve_Kp.sel(component = i_ap))**2 + abs(eigve_Kp.sel(component = i_bp))**2)*2 - 1 # equal to 1 if the fully + polarized

In [14]:
plot_eigenvectors(
    plots = [phases_K,
             phases_Kp],
    eigvas = [[eigva_K]*3,
              [eigva_Kp]*3],
    titles = [["ϕ A+/A-, K","ϕ A+/B+, K","ϕ A+/B-, K"],
              ["ϕ A+/A-, K'","ϕ A+/B+, K'","ϕ A+/B-, K'"]],
    templates=[['phase']*3,
               ['phase']*3]
)

    'data': [{'colorbar': {'len': 0.35,
                           'ticktext': […

In [15]:
plot_eigenvectors(
    plots = [[sublattice_localization_K, polarization_degree_K],
             [sublattice_localization_Kp, polarization_degree_Kp]],
    eigvas = [[eigva_K]*2,
              [eigva_Kp]*2],
    titles = [["Sublattice localization, K","Degree of polarization, K"],
              ["Sublattice localization, K'","Degree of polarization, K'"]],
    templates=[[template_sublattice, template_polarization],
               [template_sublattice, template_polarization]]
)

    'data': [{'colorbar': {'len': 0.35,
                           'ticktext': […

In [16]:
plot_eigenvectors(
    plots = [[berry_K, berry_Kp]],
    eigvas = [[eigva_K, eigva_Kp]],
    titles = [["Berry curvature, K","Berry curvature, K'"]],
    templates=[['symmetric', 'symmetric']],
)

    'data': [{'colorbar': {'len': 0.7, 'tickformat': '.0e', 'x': np.float64(0.42…